In [1]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'features_30_sec.csv', 'features_3_sec.csv', 'genres_original', 'images_original', 'Untitled.ipynb', 'Untitled1.ipynb']


In [4]:
!pip install librosa



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install tqdm



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install tensorflow


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install tensorflow



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
import librosa
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [17]:
SAMPLE_RATE = 22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def extract_mfcc(file_path, max_pad_len=1300):
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(signal, sr=sr, n_mfcc=13)

    # pad or trim to make it uniform
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]

    return mfcc.T  # shape: (1300, 13)


In [18]:
data = []
labels = []
genres = sorted(os.listdir(DATA_PATH))  # keep order consistent

for label, genre in enumerate(genres):
    genre_folder = os.path.join(DATA_PATH, genre)
    for file in tqdm(os.listdir(genre_folder), desc=f"Processing {genre}"):
        file_path = os.path.join(genre_folder, file)
        mfcc = extract_mfcc(file_path)
        data.append(mfcc)
        labels.append(label)

# Convert to numpy arrays
X = np.array(data)[..., np.newaxis]  # (samples, 1300, 13, 1)
y = tf.keras.utils.to_categorical(labels, num_classes=10)


Processing blues:   0%|                                                                        | 0/100 [00:00<?, ?it/s]


TypeError: mfcc() takes 0 positional arguments but 1 positional argument (and 2 keyword-only arguments) were given

In [19]:
import librosa
print(librosa.__file__)


C:\Users\ancye\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\__init__.py


In [1]:
import librosa
import numpy as np

file_path = "genres_original/blues/blues.00000.wav"

y, sr = librosa.load(file_path, sr=22050)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
print(mfcc.shape)


(13, 1293)


In [3]:
DATA_PATH = "genres_original"  # or your folder name


In [5]:
import os


In [6]:
import os
import librosa
import numpy as np
from tqdm import tqdm


In [8]:
SAMPLE_RATE = 22050

def extract_mfcc(file_path, max_pad_len=1300):
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)

    # Pad or trim the MFCC to max_pad_len
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]

    return mfcc.T  # Shape: (1300, 13)


In [10]:
data = []
labels = []
genres = sorted(os.listdir(DATA_PATH))

for label, genre in enumerate(genres):
    genre_folder = os.path.join(DATA_PATH, genre)
    for file in tqdm(os.listdir(genre_folder), desc=f"Processing {genre}"):
        file_path = os.path.join(genre_folder, file)
        try:
            mfcc = extract_mfcc(file_path)
            data.append(mfcc)
            labels.append(label)
        except Exception as e:
            print(f"Skipped {file_path} due to error: {e}")


Processing jazz:  54%|██████████████████████████████████▌                             | 54/100 [00:06<00:05,  8.77it/s]C:\Users\ancye\AppData\Local\Temp\ipykernel_28008\542560923.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
Processing jazz:  56%|███████████████████████████████████▊                            | 56/100 [00:06<00:04, 10.26it/s]

Skipped genres_original\jazz\jazz.00054.wav due to error: 


Processing rock: 100%|███████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  8.03it/s]


In [11]:
X = np.array(data)[..., np.newaxis]  # Shape: (samples, 1300, 13, 1)
y = tf.keras.utils.to_categorical(labels, num_classes=10)


NameError: name 'tf' is not defined

In [12]:
import tensorflow as tf


In [13]:
X = np.array(data)[..., np.newaxis]  # Shape: (samples, 1300, 13, 1)
y = tf.keras.utils.to_categorical(labels, num_classes=10)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 genres
])


C:\Users\ancye\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [17]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32
)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 318ms/step - accuracy: 0.1318 - loss: 84.5978 - val_accuracy: 0.1200 - val_loss: 2.2692
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 279ms/step - accuracy: 0.1558 - loss: 2.2256 - val_accuracy: 0.1400 - val_loss: 2.2755
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 282ms/step - accuracy: 0.1431 - loss: 2.2382 - val_accuracy: 0.1800 - val_loss: 2.2512
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 286ms/step - accuracy: 0.1912 - loss: 2.1411 - val_accuracy: 0.1950 - val_loss: 2.2278
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 313ms/step - accuracy: 0.1900 - loss: 2.0358 - val_accuracy: 0.2150 - val_loss: 2.1758
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 553ms/step - accuracy: 0.2357 - loss: 2.0170 - val_accuracy: 0.2450 - val_loss: 2.1864
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 559ms/step - accuracy: 0.2414 - loss: 1.9588 - val_accuracy: 0.2200 - val_loss: 2.2126
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 412ms/step - accuracy: 0.3174 - loss: 1.8906 - val_accurac

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.3490 - loss: 2.3989
Test Accuracy: 0.38


In [19]:
model.save("music_genre_cnn_model.h5")


In [20]:
file_path = "genres_original/pop/pop.00001.wav"


In [21]:
def predict_genre(file_path):
    mfcc = extract_mfcc(file_path)
    mfcc = mfcc[np.newaxis, ..., np.newaxis]  # (1, 1300, 13, 1)
    prediction = model.predict(mfcc)
    genre_index = np.argmax(prediction)
    genre_name = genres[genre_index]
    confidence = prediction[0][genre_index]
    return genre_name, confidence


In [22]:
genre, score = predict_genre("genres_original/jazz/jazz.00011.wav")
print(f"Predicted: {genre} ({score:.2f} confidence)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
Predicted: jazz (0.75 confidence)


In [23]:
pip install streamlit


   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   -------- ------------------------------- 2.1/10.1 MB 10.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.5/10.1 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.8/10.1 MB 9.8 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/10.1 MB 8.4 MB/s eta 0:00:01
   ------------------------------- -------- 7.9/10.1 MB 7.7 MB/s eta 0:00:01
   --------------------------------- ------ 8.4/10.1 MB 6.8 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.1 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   -------------- ------------------------- 262.1/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   - ------------


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import streamlit as st
import librosa
import numpy as np
import tensorflow as tf

SAMPLE_RATE = 22050
genres = ['blues', 'classical', 'country', 'disco', 'hiphop',
          'jazz', 'metal', 'pop', 'reggae', 'rock']

def extract_mfcc(file_path, max_pad_len=1300):
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    return mfcc.T[..., np.newaxis]

model = tf.keras.models.load_model("music_genre_cnn_model.h5")

st.title("🎧 Music Genre Classifier")

file = st.file_uploader("Upload a WAV audio file", type=["wav"])

if file is not None:
    st.audio(file, format='audio/wav')
    with open("temp.wav", "wb") as f:
        f.write(file.read())
    mfcc = extract_mfcc("temp.wav")
    mfcc = np.expand_dims(mfcc, axis=0)
    prediction = model.predict(mfcc)
    genre = genres[np.argmax(prediction)]
    confidence = np.max(prediction)
    st.success(f"🎵 Predicted Genre: **{genre}** ({confidence:.2f} confidence)")


2025-07-13 14:53:18.309 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:20.453 
  command:

    streamlit run C:\Users\ancye\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-13 14:53:20.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:20.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:20.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:20.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:20.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-13 14:53:

In [27]:
streamlit run app.py


SyntaxError: invalid syntax (507122745.py, line 1)